## 4. LangChain解説

In [ ]:
from langchain_openai import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.0)

result = llm.invoke("自己紹介してください。")
print(result)

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="こんにちは！私はジョンと言います！"),
    AIMessage(content="こんにちは、ジョンさん！どのようにお手伝いできますか？"),
    HumanMessage(content= "私の名前が分かりますか？")
]

result = chat.invoke(messages)
print(result.content)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

messages = [HumanMessage(content="自己紹介してください。")]

for chunk in chat.stream(messages):
    print(chunk.content)

In [9]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """以下のレシピを考えてください。

料理名: {dish}
"""
)

result = prompt.format(dish="カレー")
print(result)

以下のレシピを考えてください。

料理名: カレー



In [10]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "あなたは{country}料理のプロフェッショナルです。"),
    ("human", "以下の料理のレシピを考えてください。\n\n料理名: {dish}")
])

messages = chat_prompt.format_messages(country="イギリス", dish="肉じゃが")

print(messages)

[SystemMessage(content='あなたはイギリス料理のプロフェッショナルです。', additional_kwargs={}, response_metadata={}), HumanMessage(content='以下の料理のレシピを考えてください。\n\n料理名: 肉じゃが', additional_kwargs={}, response_metadata={})]


In [11]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.0)
result = chat.invoke(messages)
print(result.content)

肉じゃがは日本の家庭料理ですが、イギリス風にアレンジしたレシピをご紹介します。イギリスの食材を使いながら、肉じゃがの基本的な要素を取り入れたレシピです。

### イギリス風肉じゃが

#### 材料（4人分）
- 牛肉（薄切りまたは角切り）: 400g
- ジャガイモ: 4個（中サイズ）
- ニンジン: 2本
- 玉ねぎ: 1個
- グリーンピース: 100g（冷凍でも可）
- ビーフストック: 500ml
- ソース（ウスターソースまたはトマトケチャップ）: 大さじ2
- 砂糖: 大さじ1
- 塩: 適量
- 黒胡椒: 適量
- サラダ油: 大さじ2
- パセリ（飾り用）: 適量

#### 作り方
1. **下ごしらえ**:
   - ジャガイモは皮をむき、一口大に切ります。
   - ニンジンは皮をむき、斜め切りにします。
   - 玉ねぎは薄切りにします。

2. **肉の調理**:
   - 大きな鍋にサラダ油を熱し、牛肉を加えて中火で炒めます。肉が色づいたら、玉ねぎを加えてさらに炒めます。

3. **野菜の追加**:
   - 玉ねぎが透明になったら、ジャガイモとニンジンを加え、全体をよく混ぜます。

4. **煮込み**:
   - ビーフストックを鍋に注ぎ入れ、ウスターソースまたはトマトケチャップ、砂糖を加えます。全体を軽く混ぜ、沸騰させます。
   - 沸騰したら、火を弱めて蓋をし、約20〜30分煮込みます。ジャガイモとニンジンが柔らかくなるまで煮ます。

5. **仕上げ**:
   - 煮込みが終わったら、グリーンピースを加え、さらに5分ほど煮ます。塩と黒胡椒で味を調えます。

6. **盛り付け**:
   - 器に盛り付け、パセリを散らして完成です。

### 提案
イギリス風肉じゃがは、マッシュポテトやバターライスと一緒に楽しむと、よりボリューム感が出て美味しくいただけます。また、好みに応じて、他の野菜（例えば、セロリやパースニップ）を加えても良いでしょう。


In [12]:
from pydantic import BaseModel, Field


class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")

In [30]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "以下の料理のレシピを考えてください。\n\n料理名: {dish}")
    ]
)

chat = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.0).with_structured_output(Recipe)

chain = prompt | chat

output = chain.invoke({"dish": "カレー"})

In [31]:
print(output)

ingredients=['鶏肉', '玉ねぎ', 'にんじん', 'じゃがいも', 'カレールー', '水', 'サラダ油', '塩', 'こしょう', 'ガーリックパウダー', '生姜', 'ご飯'] steps=['鶏肉を一口大に切り、塩、こしょう、ガーリックパウダーで下味をつける。', '玉ねぎを薄切り、にんじんとじゃがいもを一口大に切る。', '鍋にサラダ油を熱し、玉ねぎを炒めて透明になるまで炒める。', '鶏肉を加え、表面が白くなるまで炒める。', 'にんじんとじゃがいもを加え、全体をよく混ぜる。', '水を加え、沸騰したらアクを取り、弱火で20分煮る。', 'カレールーを加え、さらに10分煮込む。', '味を見て、必要に応じて塩やこしょうで調整する。', 'ご飯を皿に盛り、カレーをかけて完成。']


In [32]:
output.model_dump()

{'ingredients': ['鶏肉',
  '玉ねぎ',
  'にんじん',
  'じゃがいも',
  'カレールー',
  '水',
  'サラダ油',
  '塩',
  'こしょう',
  'ガーリックパウダー',
  '生姜',
  'ご飯'],
 'steps': ['鶏肉を一口大に切り、塩、こしょう、ガーリックパウダーで下味をつける。',
  '玉ねぎを薄切り、にんじんとじゃがいもを一口大に切る。',
  '鍋にサラダ油を熱し、玉ねぎを炒めて透明になるまで炒める。',
  '鶏肉を加え、表面が白くなるまで炒める。',
  'にんじんとじゃがいもを加え、全体をよく混ぜる。',
  '水を加え、沸騰したらアクを取り、弱火で20分煮る。',
  'カレールーを加え、さらに10分煮込む。',
  '味を見て、必要に応じて塩やこしょうで調整する。',
  'ご飯を皿に盛り、カレーをかけて完成。']}

In [34]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.0)

cot_prompt = PromptTemplate.from_template(
    """以下の質問に回答してください。

質問: {question}

ステップバイステップで考えましょう。
"""
)

cot_chain = (
    {"question": RunnablePassthrough()}
    | cot_prompt
    | model
    | StrOutputParser()
)

summarize_prompt = PromptTemplate.from_template(
    """以下の文章の結論だけを一言に要約してください。

{input}"""
)

summarize_chain = (
    {"input": RunnablePassthrough()}
    | summarize_prompt
    | model
    | StrOutputParser()
)

cot_summarize_chain = cot_chain | summarize_chain

result = cot_summarize_chain.invoke(
    "私は市場に行って10個のリンゴを買いました。隣人に2つ、修理工に2つ渡しました。それから5つのリンゴを買って1つ食べました。残りは何個ですか？"
)
print(result)

残りのリンゴは10個です。


In [ ]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.memory import ConversationBufferMemory

chat = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.0)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}")
    ]
)

memory = ConversationBufferMemory(return_messages=True)

chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | prompt
    | model
    | StrOutputParser()
)

while True:
    user_message = input("You: ")
    inputs = {"input": user_message}

    ai_message = chain.invoke(inputs)
    memory.save_context(inputs, {"output": ai_message})

    print(ai_message)

こんにちは！どのようにお手伝いできますか？
森さん、こんにちは！お会いできて嬉しいです。今日はどんなことをお話ししましょうか？
あなたの名前は森さんです。何か他にお話ししたいことがありますか？
何かお手伝いできることがあれば教えてくださいね。質問や話題があればお気軽にどうぞ！
